### Importación paquetes

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import TimeSeriesSplit

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # for HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.model_selection import GridSearchCV


%config IPCompleter.greedy = True

In [33]:
ruta_proyecto = r'C:\Users\manug\Desktop\Data Science\DataSets\Wall-Mart\wallmart\\'

In [34]:
nombre_x = 'x_preseleccionado.pickle'
nombre_y = 'y_preseleccionado.pickle'

In [35]:
x = pd.read_pickle(ruta_proyecto + '02_Datos/03_Trabajo/' + nombre_x)
y = pd.read_pickle(ruta_proyecto + '02_Datos/03_Trabajo/' + nombre_y)

### Modelizar para una tienda

In [36]:
x

,date,date,store,weekly_sales,weekly_sales_mm_2_mms,weekly_sales_mm_4_mms,weekly_sales_lag_1_mms,weekly_sales_mm_8_mms,weekly_sales_lag_2_mms,weekly_sales_lag_3_mms,...,temperature_lag_2_mms,temperature_lag_1_mms,fuel_price_mm_4_mms,fuel_price_mm_2_mms,markdown1_mms,markdown5_mms,fuel_price_lag_4_mms,fuel_price_lag_3_mms,fuel_price_mms,fuel_price_lag_2_mms
0,2010-03-26,2010-03-26,1,26229.21,0.057131,0.069901,0.038857,0.102919,0.037291,0.038415,...,0.554207,0.554207,0.116992,0.118663,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
1,2010-03-26,2010-03-26,1,41892.55,0.097696,0.133360,0.069625,0.167232,0.069821,0.076476,...,0.554207,0.523581,0.118663,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
2,2010-03-26,2010-03-26,1,10366.85,0.025460,0.033576,0.020041,0.042149,0.021648,0.024731,...,0.523581,0.523581,0.120334,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.122006
3,2010-03-26,2010-03-26,1,33872.07,0.078564,0.104786,0.056020,0.131159,0.053944,0.061704,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.115320,0.122006,0.122006,0.122006
4,2010-03-26,2010-03-26,1,38971.89,0.071589,0.080878,0.039567,0.100838,0.037943,0.040211,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.122006,0.122006,0.122006,0.122006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357379,2012-07-20,2012-07-20,45,3568.64,0.012933,0.016792,0.013349,0.018471,0.016531,0.013535,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357380,2012-07-20,2012-07-20,45,3618.52,0.010426,0.012421,0.009989,0.013434,0.012398,0.012430,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357381,2012-07-20,2012-07-20,45,53136.34,0.123414,0.171109,0.087250,0.209947,0.096830,0.089377,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187
357382,2012-07-20,2012-07-20,45,6902.68,0.019104,0.023781,0.016667,0.027779,0.018808,0.016799,...,0.792074,0.792074,0.587187,0.587187,0.075386,0.021087,0.587187,0.587187,0.587187,0.587187


In [38]:
x = x.loc[x.store == 1]

In [39]:
x

,date,date,store,weekly_sales,weekly_sales_mm_2_mms,weekly_sales_mm_4_mms,weekly_sales_lag_1_mms,weekly_sales_mm_8_mms,weekly_sales_lag_2_mms,weekly_sales_lag_3_mms,...,temperature_lag_2_mms,temperature_lag_1_mms,fuel_price_mm_4_mms,fuel_price_mm_2_mms,markdown1_mms,markdown5_mms,fuel_price_lag_4_mms,fuel_price_lag_3_mms,fuel_price_mms,fuel_price_lag_2_mms
0,2010-03-26,2010-03-26,1,26229.21,0.057131,0.069901,0.038857,0.102919,0.037291,0.038415,...,0.554207,0.554207,0.116992,0.118663,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
1,2010-03-26,2010-03-26,1,41892.55,0.097696,0.133360,0.069625,0.167232,0.069821,0.076476,...,0.554207,0.523581,0.118663,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.115320
2,2010-03-26,2010-03-26,1,10366.85,0.025460,0.033576,0.020041,0.042149,0.021648,0.024731,...,0.523581,0.523581,0.120334,0.122006,0.000000,0.000000,0.115320,0.115320,0.122006,0.122006
3,2010-03-26,2010-03-26,1,33872.07,0.078564,0.104786,0.056020,0.131159,0.053944,0.061704,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.115320,0.122006,0.122006,0.122006
4,2010-03-26,2010-03-26,1,38971.89,0.071589,0.080878,0.039567,0.100838,0.037943,0.040211,...,0.523581,0.523581,0.122006,0.122006,0.000000,0.000000,0.122006,0.122006,0.122006,0.122006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8700,2012-07-20,2012-07-20,1,126861.49,0.284544,0.397458,0.192975,0.487956,0.220297,0.196142,...,0.807045,0.807045,0.444568,0.444568,0.036245,0.029748,0.444568,0.444568,0.444568,0.444568
8701,2012-07-20,2012-07-20,1,37535.69,0.082387,0.120271,0.055787,0.146633,0.074215,0.066627,...,0.807045,0.807045,0.444568,0.444568,0.036245,0.029748,0.444568,0.444568,0.444568,0.444568
8702,2012-07-20,2012-07-20,1,34958.10,0.084541,0.114040,0.062304,0.138966,0.070169,0.055693,...,0.807045,0.807045,0.444568,0.444568,0.036245,0.029748,0.444568,0.444568,0.444568,0.444568
8703,2012-07-20,2012-07-20,1,11847.91,0.029826,0.036914,0.023645,0.043973,0.025981,0.021184,...,0.807045,0.807045,0.444568,0.444568,0.036245,0.029748,0.444568,0.444568,0.444568,0.444568


In [40]:
y = y[y.index.isin(x.index)]

In [41]:
var_modelizar = x.columns.to_list()[2:]

### Definir la validación

In [42]:
time_cv = TimeSeriesSplit(3, test_size = 8)

### Crear el pipe y el diccionario de algorimos, parámetros y valores a testar

In [43]:
# Definición del Pipeline
pipe = Pipeline([('algoritmo', LinearRegression())])

# Definición de los hiperparámetros a evaluar
grid = [
    {'algoritmo': [LinearRegression()]},


    {'algoritmo': [HistGradientBoostingRegressor()],
     'algoritmo__learning_rate': [0.01, 0.025, 0.05, 0.1],
     'algoritmo__max_iter': [50, 100, 200],
     'algoritmo__max_depth': [5, 10, 20],
     'algoritmo__min_samples_leaf': [500],
     'algoritmo__l2_regularization': [0, 0.25, 0.5, 0.75, 1]}
]

In [44]:
grid_search = GridSearchCV(estimator= pipe, 
                            param_grid = grid, 
                            cv = time_cv, 
                            scoring = 'neg_mean_squared_error',
                            verbose = 0,
                            n_jobs = -1)

modelo = grid_search.fit(x[var_modelizar],y)

pd.DataFrame(grid_search.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algoritmo,param_algoritmo__l2_regularization,param_algoritmo__learning_rate,param_algoritmo__max_depth,param_algoritmo__max_iter,param_algoritmo__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035586,0.000849,0.002990,8.144910e-04,LinearRegression(),NaN,NaN,NaN,NaN,NaN,{'algoritmo': LinearRegression()},-4.444060e-22,-3.879994e-23,-2.023560e-21,-8.355888e-22,8.561879e-22,1
30,0.623020,0.054123,0.003989,8.143935e-04,HistGradientBoostingRegressor(),0,0.1,5,200,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.216795e+05,-1.239312e+07,-1.104127e+08,-4.097582e+07,4.935419e+07,2
108,0.777586,0.039896,0.003990,8.139067e-04,HistGradientBoostingRegressor(),0.5,0.1,20,200,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.091565e+05,-1.231215e+07,-1.110451e+08,-4.115548e+07,4.966992e+07,3
72,0.780911,0.042439,0.003989,8.148803e-04,HistGradientBoostingRegressor(),0.25,0.1,20,200,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-9.088960e+04,-1.236258e+07,-1.110177e+08,-4.115707e+07,4.965235e+07,4
105,0.809169,0.020315,0.003657,4.701903e-04,HistGradientBoostingRegressor(),0.5,0.1,10,200,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.109107e+05,-1.234063e+07,-1.110447e+08,-4.116542e+07,4.966372e+07,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.199466,0.011828,0.002327,4.701341e-04,HistGradientBoostingRegressor(),0.5,0.01,5,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-9.739215e+07,-2.957646e+08,-1.618623e+09,-6.705934e+08,6.752326e+08,177
109,0.194812,0.007389,0.001995,4.899036e-07,HistGradientBoostingRegressor(),0.75,0.01,5,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-9.745552e+07,-2.959922e+08,-1.619175e+09,-6.708744e+08,6.754309e+08,178
148,0.212099,0.014042,0.002327,4.695721e-04,HistGradientBoostingRegressor(),1,0.01,10,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-9.707107e+07,-2.961421e+08,-1.619504e+09,-6.709057e+08,6.756658e+08,179
151,0.223070,0.004909,0.001995,1.946680e-07,HistGradientBoostingRegressor(),1,0.01,20,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-9.707107e+07,-2.961421e+08,-1.619504e+09,-6.709057e+08,6.756658e+08,179


In [46]:
resultados

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algoritmo,param_algoritmo__l2_regularization,param_algoritmo__learning_rate,param_algoritmo__max_depth,param_algoritmo__max_iter,param_algoritmo__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018285,0.000941,0.001329,4.703026e-04,LinearRegression(),NaN,NaN,NaN,NaN,NaN,{'algoritmo': LinearRegression()},-1.705364e-21,-4.962076e-21,-7.788346e-21,-4.818595e-21,2.485439e-21,1
6,0.182512,0.011488,0.002660,9.400996e-04,HistGradientBoostingRegressor(),0,0.1,5,100,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.753860e+05,-2.071629e+07,-1.526606e+08,-5.785076e+07,6.756311e+07,2
8,0.258641,0.016068,0.002327,4.700779e-04,HistGradientBoostingRegressor(),0,0.1,10,100,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.866985e+05,-2.091715e+07,-1.526803e+08,-5.792805e+07,6.753237e+07,3
16,0.231048,0.007566,0.001995,5.840039e-07,HistGradientBoostingRegressor(),0.5,0.1,10,100,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.539447e+05,-2.095033e+07,-1.530958e+08,-5.806669e+07,6.772995e+07,4
14,0.200795,0.011697,0.002327,4.701903e-04,HistGradientBoostingRegressor(),0.5,0.1,5,100,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-1.832117e+05,-2.110162e+07,-1.532141e+08,-5.816630e+07,6.774931e+07,5
5,0.109041,0.002488,0.001330,4.700217e-04,HistGradientBoostingRegressor(),0,0.1,5,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-2.652439e+05,-2.630047e+07,-1.983759e+08,-7.498055e+07,8.789871e+07,6
7,0.132313,0.008157,0.001995,4.052337e-07,HistGradientBoostingRegressor(),0,0.1,10,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-2.626496e+05,-2.630255e+07,-1.984662e+08,-7.501046e+07,8.794129e+07,7
13,0.150265,0.022424,0.003324,1.243773e-03,HistGradientBoostingRegressor(),0.5,0.1,5,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-2.437902e+05,-2.650111e+07,-1.989709e+08,-7.523859e+07,8.814616e+07,8
15,0.140291,0.012935,0.002327,4.701903e-04,HistGradientBoostingRegressor(),0.5,0.1,10,50,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-2.396757e+05,-2.670194e+07,-1.988144e+08,-7.525200e+07,8.803715e+07,9
12,0.263296,0.017635,0.003989,1.410515e-03,HistGradientBoostingRegressor(),0.5,0.01,10,100,500,"{'algoritmo': HistGradientBoostingRegressor(),...",-3.605085e+07,-7.356739e+07,-8.927911e+08,-3.341365e+08,3.953253e+08,10


In [23]:
x

,date,store,weekly_sales,dept,weekly_sales_mm_2,weekly_sales_mm_4,weekly_sales_lag_1,weekly_sales_mm_8,weekly_sales_lag_2,weekly_sales_lag_3,...,type_te,type_C,type_A,cpi_mm_4,cpi_mm_8,cpi_mm_2,cpi_lag_4,cpi_lag_1,cpi_lag_3,cpi_lag_2
0,2010-03-26,1,26229.21,1,24182.925,22809.2850,22136.64,27900.02750,21043.39,21827.90,...,20259.231187,0.0,1.0,211.166237,211.190936,211.116839,211.215635,211.215635,211.215635,211.215635
1,2010-03-26,1,41892.55,2,42754.020,44414.4900,43615.49,45645.96625,43751.94,48397.98,...,20259.231187,0.0,1.0,211.116839,211.166237,211.018042,211.215635,211.018042,211.215635,211.215635
2,2010-03-26,1,10366.85,3,9684.110,10441.8125,9001.37,11131.73125,10123.45,12275.58,...,20259.231187,0.0,1.0,211.067441,211.141538,211.018042,211.215635,211.018042,211.215635,211.018042
3,2010-03-26,1,33872.07,4,33995.090,34686.2600,34118.11,35692.17500,32668.67,38086.19,...,20259.231187,0.0,1.0,211.018042,211.116839,211.018042,211.215635,211.018042,211.018042,211.018042
4,2010-03-26,1,38971.89,5,30802.230,26546.3800,22632.57,27325.73375,21498.92,23082.14,...,20259.231187,0.0,1.0,211.018042,211.092140,211.018042,211.018042,211.018042,211.018042,211.018042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8700,2012-07-20,1,126861.49,95,128293.115,134329.9600,129724.74,134143.88375,148798.05,131935.56,...,20259.231187,0.0,1.0,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727
8701,2012-07-20,1,37535.69,96,35745.470,39958.2000,33955.25,39962.10250,46819.51,41522.35,...,20259.231187,0.0,1.0,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727
8702,2012-07-20,1,34958.10,97,36731.445,37836.9075,38504.79,37846.61375,43995.15,33889.59,...,20259.231187,0.0,1.0,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727
8703,2012-07-20,1,11847.91,98,11682.785,11578.2525,11517.66,11634.92000,13148.00,9799.44,...,20259.231187,0.0,1.0,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727,221.932727
